<a href="https://colab.research.google.com/github/TirmidziOthman/Vehicle-Routing-Problem---ACO/blob/main/aco_tcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import folium
from shapely.geometry import Point, Polygon
import numpy as np

In [ ]:
def display_map_with_route(route):
    city_locations = {
        'Kuala Lumpur': [3.1319, 101.6841],
        'Batu Cave': [3.2379, 101.6840],
        'Petaling jaya': [3.1279, 101.5945],
        'Ampang Jaya': [3.1491, 101.7625]
    }

    map1 = folium.Map(location=[3.1319, 101.6841], zoom_start=9, prefer_canvas=True)

    for i, city in enumerate(route[:-1]):
        location = city_locations[city]
        folium.Marker(location=location, popup=str(i+1)).add_to(map1)

    folium.PolyLine(locations=[city_locations[city] for city in route],
                    weight=5,
                    color="blue",
                    line_opacity=0.5).add_to(map1)

    min_lat = min([location[0] for location in city_locations.values()])
    max_lat = max([location[0] for location in city_locations.values()])
    min_lon = min([location[1] for location in city_locations.values()])
    max_lon = max([location[1] for location in city_locations.values()])
    bounds = [[min_lat, min_lon], [max_lat, max_lon]]

    map1.fit_bounds(bounds)
    return map1

In [ ]:
route = ['Kuala Lumpur', 'Batu Cave', 'Petaling jaya', 'Ampang Jaya', 'Kuala Lumpur']
map_with_route = display_map_with_route(route)
map_with_route

In [ ]:
#Ant_count: Numbers of ants that will be exploring the population
#Iterations: Number of generations for which ants will be searching for solutions
#PheromoneConstant: The constant used as a sufficient while calculating strength of pheromone applied by an individual ant based on its tour length
#DecayConstant: Strength with which pheromone decays in a path
#Alpha: Pheromone strength while finding the next city
#Beta: Heuristic strength while finding the next city
#Initialization: How to initialize first population, either random or with a specific city


In [ ]:
def ant_colony_optimization(distance_matrix, iteration=100, n_ants=5, n_cities=5, evaporation_rate=0.5, alpha=1, beta=2):
    # Initialization
    d = distance_matrix
    m = n_ants
    n = n_cities
    e = evaporation_rate

    # Calculating visibility of the next city
    visibility = 1 / d
    visibility[visibility == np.inf] = 0

    # Initializing pheromone present at the paths to the cities
    pheromone = 0.1 * np.ones((m, n))

    # Initializing the routes of the ants
    routes = np.ones((m, n + 1))

    for ite in range(iteration):
        routes[:, 0] = 1

        for i in range(m):
            temp_visibility = np.array(visibility)

            for j in range(n - 1):
                combine_feature = np.zeros(n)
                cum_prob = np.zeros(n)

                cur_loc = int(routes[i, j] - 1)
                temp_visibility[:, cur_loc] = 0

                p_feature = np.power(pheromone[cur_loc, :], beta)
                v_feature = np.power(temp_visibility[cur_loc, :], alpha)

                p_feature = p_feature[:, np.newaxis]
                v_feature = v_feature[:, np.newaxis]

                combine_feature = np.multiply(p_feature, v_feature)
                total = np.sum(combine_feature)
                probs = combine_feature / total
                cum_prob = np.cumsum(probs)

                r = np.random.random_sample()
                city = np.nonzero(cum_prob > r)[0][0] + 1

                routes[i, j + 1] = city

            left = list(set([i for i in range(1, n + 1)]) - set(routes[i, :-2]))[0]
            routes[i, -2] = left

        optimal_route = np.array(routes)
        dist_cost = np.zeros((m, 1))

        for i in range(m):
            s = 0
            for j in range(n - 1):
                s += d[int(optimal_route[i, j]) - 1, int(optimal_route[i, j + 1]) - 1]#calcualting total tour distance
            dist_cost[i] = s

        min_loc = np.argmin(dist_cost)
        min_cost = dist_cost[min_loc]
        best_route = routes[min_loc, :]
        pheromone = (1 - e) * pheromone
        print(best_route)

        for i in range(m):
            for j in range(n - 1):
                dt = 1 / dist_cost[i]
                pheromone[int(optimal_route[i, j]) - 1, int(optimal_route[i, j + 1]) - 1] += dt
    print(visibility)

    return best_route, int(min_cost[0]) + d[int(best_route[-2]) - 1, 0],

In [ ]:
#the distances are collected from google map manually
d = np.array(
    [
        [0, 26, 16, 12],      # Tampere distances to other cities
        [26, 0, 17, 35],      # Vesilahti distances to other cities
        [16, 17, 0, 31],      # Toijala distances to other cities
        [12, 35, 31, 0],      # Sastamala distances to other cities
    ]
        [0, 26, 16, 12],      # Tampere distances to other cities
        [26, 0, 17, 35],      # Vesilahti distances to other cities
        [16, 17, 0, 31],      # Toijala distances to other cities
        [12, 35, 31, 0],      # Sastamala distances to other cities

)

iteration = 100
n_ants = 20
n_cities = 4
evaporation_rate = 0.5
alpha = 1
beta = 2

best_route, min_cost = ant_colony_optimization(d, iteration, n_ants, n_cities, evaporation_rate, alpha, beta)

print('Best route:', best_route)

<ipython-input-31-b7433ee0fea2>:9: RuntimeWarning: divide by zero encountered in true_divide
  visibility = 1 / d


[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1.]
[1. 4. 3. 2. 1